In [5]:
import argparse
import sys
from pathlib import Path
from sklearn.model_selection import train_test_split

In [6]:
import torch
from torch.utils.data import random_split
from torch.utils.data import DataLoader
import torch.nn as nn

In [11]:
# local libs
from dataset import AllVertices
from src.nn_model import AmberNN
from src.logger import Logger
from run_opts import config_runtime


ModuleNotFoundError: No module named 'dataset'

In [8]:
_ = torch.manual_seed(142)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [9]:
n_cls = 3  # number of classes
conf_dev = config_runtime['device']
train_f = config_runtime['train_frac']
seed = config_runtime['seed']
learning_rate = config_runtime['learning_rate']
batch_size = config_runtime['batch_size']
hid_size = config_runtime['hidden_size']
log_step = config_runtime['log_step']
epochs = config_runtime['num_epochs']
run_name = config_runtime['run_name']

NameError: name 'config_runtime' is not defined

In [6]:
logger = Logger("./", "train")

# store parameters
logger.print("Parameters")
logger.print("-------------------------")
logger.print(f"Train fraction:    {train_f}")
logger.print(f"Learning rate:     {learning_rate}")
logger.print(f"Batch size:        {batch_size}")
logger.print(f"Hidden layer size: {hid_size}")
logger.print(f"Number of epochs:  {epochs}")
logger.print("-------------------------")

Parameters
-------------------------
Train fraction:    0.85
Learning rate:     0.0001
Batch size:        128
Hidden layer size: 256
Number of epochs:  12
-------------------------


In [10]:
# Read list of the proteins
proteins = []
with open("../data/lists/charge_v2.txt", 'r') as iFile:
    for i in iFile:
        if i[0] != '#':
            proteins.append(i.strip())

In [11]:
train_prots, test_prots = train_test_split(proteins, train_size=train_f, random_state=42)

In [12]:
logger.print("\n")
logger.print("Data")
logger.print("-------------------------")
logger.print(f"Proteins: train {len(train_prots)}   test {len(test_prots)}")



Data
-------------------------
Proteins: train 26   test 5


In [15]:
# pd = AllVertices(proteins)
# n_features = pd[0][0].shape[0]
# train_dataset, test_dataset = random_split(pd, [train_f, 1.0 - train_f],
#                                            generator=torch.Generator().manual_seed(seed))
train_dataset = AllVertices(train_prots)
test_dataset = AllVertices(test_prots)
n_features = train_dataset[0][0].shape[0]

In [17]:
logger.print(f"Vertices: train {len(train_dataset)}   test {len(test_dataset)}")
logger.print(f"Features: {n_features}")

Vertices: train 279749   test 29564
Features: 111


In [ ]:
# test_dataset[2]

In [ ]:
# Create data loaders.
train_dataloader = DataLoader(train_dataset, batch_size=batch_size)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size)

model = AmberNN(n_features, n_cls, hid_size).to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

NameError: name 'train_dataset' is not defined

In [ ]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        X = X.to(device)
        y = y.to(device)
        pred = model(X)
        loss = loss_fn(pred, y)
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        sys.stdout.write(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]\n")

In [ ]:
X = torch.stack([train_dataset[333][0], train_dataset[235][0]], axis=0)
X = X.to(device)

In [ ]:
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

In [ ]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X,y) in enumerate(dataloader):
        # Compute prediction and loss
        X = X.to(device)
        y = y.to(device)
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 1000 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            
            X = X.to(device)
            y = y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
        test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [ ]:
epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

In [ ]:
torch.save(model.state_dict(), "20230301_model.pytorch")

## Test the model

In [ ]:
Trial_pdb = "6BOY_B"

In [ ]:
data = ProteinRecord(Trial_pdb)

In [ ]:
X = data.f.to(device)
p = data.p.to(device)
y = data.y_aux.to(device)

In [ ]:
with torch.no_grad():
    logits = model(X)
    pred_probab = nn.Softmax(dim=1)(logits)
    y_pred = pred_probab.argmax(1)
    # print(f"Predicted class: {y_pred}")

In [ ]:
save_for_paraview(f"predicted_{Trial_pdb}.csv", p, y_pred)
save_for_paraview(f"real_{Trial_pdb}.csv", p, y)
        

In [ ]:
y[580:680]